In [7]:
# MINLP.ipynb
#
# Versão 2.3 - Demanda Viável
# - 48 períodos de 30 minutos
# - Restrições de Rampa
# - Restrições de Tempo Mínimo de Operação (Min Uptime)
import gurobipy as gp
from gurobipy import GRB
import time

# --- 1. Definição dos Dados do Problema (v2.3) ---

# 48 períodos de 30 minutos
T = range(1, 49)

# Demanda por período (MW)
D_lista = [
    90,
    98, 95, 93, 90, 90, 92, 95, 100, 105, 110, 120, 130, 140, 150, 165, 
    180, 195, 210, 225, 240, 245, 250, 248, 245, 240, 230, 225, 220, 215, 
    210, 205, 200, 195, 190, 200, 210, 220, 230, 240, 235, 230, 215, 200, 
    175, 150, 130, 115
]
D = {t: D_lista[t-1] for t in T}

# --- Usina a Gás (Linear) ---
gas_params = {
    'var_cost': 80,    # R$/MW
    'fix_cost': 250,   # R$/período (R$500/hr)
    'max_cap': 150,    # MW
    'min_gen': 45,     # MW (30% de 150)
    'ramp_rate': 50,   # MW/30min
    'min_uptime': 3    # períodos (1.5 horas)
}

# --- Usina a Carvão (Não-Linear) ---
carvao_params = {
    # Custo Variável: 60*P + 0.2*P^2
    'var_cost_linear': 60,    # R$/MW
    'var_cost_quad': 0.2,     # R$/MW^2
    'fix_cost': 1000,         # R$/período (R$2000/hr)
    'max_cap': 200,           # MW
    'min_gen': 30,            # MW (15% de 200)
    'ramp_rate': 40,          # MW/30min
    'min_uptime': 1           # períodos (30 min)
}

# --- 2. Criação do Modelo ---
m = gp.Model("MINLP_Despacho_v2_Viável")

# --- 3. Definição das Variáveis de Decisão ---
T_with_0 = range(0, 49) 
P_gas = m.addVars(T_with_0, name="P_gas", lb=0.0)
P_carvao = m.addVars(T_with_0, name="P_carvao", lb=0.0)
z_gas = m.addVars(T_with_0, name="z_gas", vtype=GRB.BINARY)
z_carvao = m.addVars(T_with_0, name="z_carvao", vtype=GRB.BINARY)
start_gas = m.addVars(T, name="start_gas", vtype=GRB.BINARY)
start_carvao = m.addVars(T, name="start_carvao", vtype=GRB.BINARY)

# --- 4. Definição da Função Objetivo (MIQCP) ---
custo_gas = gp.quicksum(
    gas_params['var_cost'] * P_gas[t] + gas_params['fix_cost'] * z_gas[t]
    for t in T
)

custo_carvao = gp.quicksum(
    carvao_params['var_cost_linear'] * P_carvao[t] +
    carvao_params['var_cost_quad'] * P_carvao[t] * P_carvao[t] +
    carvao_params['fix_cost'] * z_carvao[t]
    for t in T
)

m.setObjective(custo_gas + custo_carvao, GRB.MINIMIZE)

# --- 5. Definição das Restrições ---

# Estado Inicial (t=0)
m.addConstr(P_gas[0] == 0, name="Gas_Initial_P")
m.addConstr(z_gas[0] == 0, name="Gas_Initial_z")
m.addConstr(P_carvao[0] == 0, name="Carvao_Initial_P")
m.addConstr(z_carvao[0] == 0, name="Carvao_Initial_z")

# Restrições para cada período t = 1 a 48
for t in T:
    
    # 1. Atender Demanda (Relaxado para '>=')
    m.addConstr(
        P_gas[t] + P_carvao[t] >= D[t],
        name=f"Demanda_t{t}"
    )

    # 2. Restrições da Usina a Gás (Geração)
    m.addConstr(
        P_gas[t] <= gas_params['max_cap'] * z_gas[t],
        name=f"Gas_MaxCap_t{t}"
    )
    m.addConstr(
        P_gas[t] >= gas_params['min_gen'] * z_gas[t],
        name=f"Gas_MinGen_t{t}"
    )

    # 3. Restrições da Usina a Carvão (Geração)
    m.addConstr(
        P_carvao[t] <= carvao_params['max_cap'] * z_carvao[t],
        name=f"Carvao_MaxCap_t{t}"
    )
    m.addConstr(
        P_carvao[t] >= carvao_params['min_gen'] * z_carvao[t],
        name=f"Carvao_MinGen_t{t}"
    )

    # 4. Restrições de Rampa
    # Gás
    m.addConstr(
        P_gas[t] - P_gas[t-1] <= gas_params['ramp_rate'],
        name=f"Gas_RampUp_t{t}"
    )
    m.addConstr(
        P_gas[t-1] - P_gas[t] <= gas_params['ramp_rate'],
        name=f"Gas_RampDown_t{t}"
    )
    # Carvão
    m.addConstr(
        P_carvao[t] - P_carvao[t-1] <= carvao_params['ramp_rate'],
        name=f"Carvao_RampUp_t{t}"
    )
    m.addConstr(
        P_carvao[t-1] - P_carvao[t] <= carvao_params['ramp_rate'],
        name=f"Carvao_RampDown_t{t}"
    )

    # 5. Restrições de Partida
    m.addConstr(
        start_gas[t] >= z_gas[t] - z_gas[t-1],
        name=f"Gas_StartDef_t{t}"
    )
    m.addConstr(
        start_carvao[t] >= z_carvao[t] - z_carvao[t-1],
        name=f"Carvao_StartDef_t{t}"
    )

    # 6. Restrições de Tempo Mínimo de Operação (Min Uptime)
    # Gás: k = 3 períodos
    k_gas = gas_params['min_uptime']
    for tau in range(t, min(t + k_gas, 49)): 
        m.addConstr(
            z_gas[tau] >= start_gas[t],
            name=f"Gas_MinUptime_t{t}_tau{tau}"
        )

    # Carvão: k = 1 período
    k_carvao = carvao_params['min_uptime']
    for tau in range(t, min(t + k_carvao, 49)):
        m.addConstr(
            z_carvao[tau] >= start_carvao[t],
            name=f"Carvao_MinUptime_t{t}_tau{tau}"
        )

# --- 6. Otimização do Modelo ---
print("--- Iniciando Otimização do Modelo MINLP v2.3 (Benchmark) ---")
m.setParam('Presolve', 0)
start_time = time.time()

m.optimize()
end_time = time.time()
print("--- Otimização Concluída ---")

solve_time = end_time - start_time

# --- 7. Exibição dos Resultados ---
if m.Status == GRB.OPTIMAL:
    print(f"\nSolução Ótima Encontrada!")
    print(f"Custo Total (Benchmark): R$ {m.ObjVal:.2f}")
    print(f"Tempo de Solução: {solve_time:.4f} segundos")
    print("-" * 70)
    
    print("Resumo da Operação (Geração em MW):")
    print("Per. | Demanda | Gás (P)   | Gás (z) | Carvão (P) | Carvão (z) | Total Gen. | Sobra")
    print("-" * 70)
    for t in T:
        gas_gen = P_gas[t].X
        gas_z = "ON" if z_gas[t].X > 0.5 else "OFF"
        carvao_gen = P_carvao[t].X
        carvao_z = "ON" if z_carvao[t].X > 0.5 else "OFF"
        total_gen = gas_gen + carvao_gen
        sobra = total_gen - D[t]
        
        print(f" {t:2d} |  {D[t]:3d}  | {gas_gen:8.2f} | {gas_z:^7} | {carvao_gen:10.2f} | {carvao_z:^10} | {total_gen:10.2f} | {sobra:5.2f}")

elif m.Status == GRB.INFEASIBLE:
    print("\n--- ERRO: Modelo Inviável (Mesmo após relaxamento) ---")
    m.computeIIS()
    m.write("modelo_conflito.ilp")
    print("IIS salvo em 'modelo_conflito.ilp'. Verifique os dados de demanda vs min/max gen.")
else:
    print(f"Otimização terminou com status: {m.Status}")

--- Iniciando Otimização do Modelo MINLP v2.3 (Benchmark) ---
Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Presolve  0

Optimize a model with 721 rows, 292 columns and 1534 nonzeros
Model fingerprint: 0x3e8c8f75
Model has 48 quadratic objective terms
Variable types: 98 continuous, 194 integer (194 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [6e+01, 1e+03]
  QObjective range [4e-01, 4e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 3e+02]
Found heuristic solution: objective 889685.00000
Variable types: 96 continuous, 196 integer (194 binary)

Root relaxation: objective 7.032121e+05, 241 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objec